In [1]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [3]:
# seed 값 설정
seed = 0

np.random.seed(seed)
tf.random.set_seed(3)

In [4]:
# 데이터 불러오기
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype("float32") / 255.0
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype("float32") / 255.0
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

11493376/11490434 [==============================] - 2s 0us/step


In [7]:
# 컨볼루션 신경망 설정
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), input_shape = (28, 28, 1), activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

In [8]:
model.compile(loss = "categorical_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])

In [9]:
# 모델 최적화 설정
MODEL_DIR = "./model/"

if not os.path.exists(MODEL_DIR):
  os.mkdir(MODEL_DIR)

In [10]:
modelpath = "./{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath = modelpath, monitor = "val_loss", verbose = 1, save_best_only=True)

In [ ]:
early_stopping_callback = EarlyStopping(monitor = "val_loss", patience=10)

In [ ]:
# 모델의 실행
history = model.fit(X_train, Y_train, validation_data = (X_test, Y_test), epochs = 30, batch_size= 200, verbose = 0, callbacks = [early_stopping_callback, checkpointer])


Epoch 00001: val_loss improved from inf to 0.05626, saving model to ./01-0.0563.hdf5

Epoch 00002: val_loss improved from 0.05626 to 0.03958, saving model to ./02-0.0396.hdf5

Epoch 00003: val_loss improved from 0.03958 to 0.03402, saving model to ./03-0.0340.hdf5

Epoch 00004: val_loss did not improve from 0.03402

Epoch 00005: val_loss improved from 0.03402 to 0.03121, saving model to ./05-0.0312.hdf5


KeyboardInterrupt: ignored

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuaracy : %.4f" % (model.evaluate(X_test, Y_test)[1]))

In [ ]:
# 테스트셋의 오차
y_vloss = history.history["val_loss"]

In [ ]:
# 학습셋의 오차
y_loss = history.history["loss"]

In [ ]:
# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker = ".", c = "red", label = "Testset_loss")
plt.plot(x_len, y_loss, marker = ".", c = "blue", label = "Trainset_loss")

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc = "upper right")
plt.grid()
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()